<a href="https://colab.research.google.com/github/MonishaO/BizCardX-Extracting-Business-Card-Data-with-OCR/blob/main/BizCardX_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opencv-python
!pip install pytesseract
!pip install easyocr
!pip install streamlit -q
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.9/813.9 kB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 14.0 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.7.0.72
    Uninstalling opencv-python-headless-4.7.0.72:
      Successfully uninstalled opencv-python-headless-4.7.0.72
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 7.5 MB/s eta 0:00:

In [2]:
!pip install --upgrade pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#!./ngrok authtokens 2OMoT6Hz0dpsfiFyJKtKPyHcZI3_4GkNi1zWuURL2uyPUQfXy

In [4]:
## For accessing app via link
#from pyngrok import ngrok 
#public_p_url = ngrok.connect(port='8501')

In [5]:
%%writefile streamlit_app.py

import streamlit as st
import easyocr
import sqlite3
import cv2
import numpy as np 
import pandas as pd


# Connect to sqlite3 database
conn = sqlite3.connect('card_info.db')
c = conn.cursor()


# Create a table to store the business card information
c.execute('''CREATE TABLE IF NOT EXISTS Business_card 
              (id INT AUTO_INCREMENT PRIMARY KEY,
              name TEXT,
              position TEXT,
              address TEXT,
              pincode VARCHAR(25),
              phone VARCHAR(25),
              email TEXT,
              website TEXT,
              company TEXT
              )''')

# Using easyOCR for reading data
reader = easyocr.Reader(['en'])

# Set the title and page icon
st.set_page_config(page_title="BIZCARDX", page_icon=":card_index:")

# Title 
st.title(":blue[Extracting Business Card Data:card_index_dividers:]")
st.title("\n\r")

# Create a file uploader
file_upload = st.file_uploader(":green[UPLOAD CARD IMAGE>>>:credit_card:]", type=["jpg", "jpeg", "png", "tiff", "tif", "gif"])


# Create a sidebar menu with options to Add, Show, Update business card information
st.sidebar.title(":blue[BIZCARDX:card_file_box:]")
st.title("\n\r")
st.title("\n\r")
st.sidebar.image("https://miro.medium.com/v2/resize:fit:1400/0*V7NS3dvYQLVi6DFL.gif", width=400)
st.title("\n\r")


m = ['INSERT DATA', 'SHOW DATA', 'EDIT CARD INFO', "DELETE DATA"]
choose = st.sidebar.selectbox(":black[SELECT AN OPTION:]", m)

if choose == 'INSERT DATA':
    if file_upload is not None:
        
        # Read the image using OpenCV
        image = cv2.imdecode(np.fromstring(file_upload.read(), np.uint8), 1)
        
        # Display the uploaded image
        st.image(image, caption='UPLOADED SUCCESSFULLY', use_column_width=True)
        
        # Button to extract information from the image
        if st.button('Extract Data And Insert'):
            bsc = reader.readtext(image, detail=0)
            text = "\n".join(bsc)
            
            # Insert the extracted information and image into the database
            sql_data = "INSERT INTO Business_card (name, position, address, pincode, phone, email, website, company) VALUES ( ?, ?, ?, ?, ?, ?, ?, ?)"
            values = (bsc[0], bsc[1], bsc[2], bsc[3], bsc[4], bsc[5], bsc[6], bsc[7])
            c.execute(sql_data, values)
            conn.commit()
            
            # Display message
            st.success("DATA INSERTED SUCCESSFULLY")

elif choose == 'SHOW DATA':
    
    # Display the stored business card information
    c.execute("SELECT * FROM Business_card")
    result = c.fetchall()
    df = pd.DataFrame(result, columns=['id','name', 'position', 'address', 'pincode', 'phone', 'email', 'website', 'company'])
    st.write(df)


elif choose == 'EDIT CARD INFO':
    
    # Create a dropdown menu to select a business card to edit
    c.execute("SELECT id, name FROM Business_card")
    result = c.fetchall()
    business_cards = {}
    
    for row in result:
        business_cards[row[1]] = row[0]
    select_card_name = st.selectbox("Select Card To Edit", list(business_cards.keys()))
    
    # Get the current information for the selected business card
    c.execute("SELECT * FROM Business_card WHERE name=?", (select_card_name,))
    result = c.fetchone()

    # Get edited information 
    name = st.text_input("Name", result[1])
    position = st.text_input("Position", result[2])
    address = st.text_input("Address", result[3])
    pincode = st.text_input("Pincode", result[4])
    phone = st.text_input("Phone", result[5])
    email = st.text_input("Email", result[6])
    website = st.text_input("Website", result[7])
    company = st.text_input("Company_Name", result[8])

    
    # Create a button to update the business card
    if st.button("Edit Card Data"):
        
        # Update the information for the selected business card in the database
        c.execute("UPDATE Business_card SET name=?, position=?, address=?, pincode=?, phone=?, email=?, website=?, company=? WHERE name=?", 
                             (name, position, address, pincode, phone, email, website, company, select_card_name))
        conn.commit()
        st.success("CARD DATA UPDATED SUCCESSFULLY")

if choose == 'DELETE DATA':
    
    # Create a dropdown menu to select a business card to delete
    c.execute("SELECT id, name FROM Business_card")
    result = c.fetchall()
    business_cards = {}
    
    for row in result:
        business_cards[row[1]] = row[0]
    select_card_name = st.selectbox("Select Card To Delete", list(business_cards.keys()))
    
    # Create a button to delete the selected business card
    if st.button("Delete Card"):
        
        # Delete the selected business card from the database
        c.execute("DELETE FROM Business_card WHERE name=?", (select_card_name,))
        conn.commit()
        st.success("CARD DATA DELETED SUCCESSFULLY")

Writing streamlit_app.py


In [ ]:
!streamlit run /content/streamlit_app.py & npx localtunnel -p 8501

[..................] | fetchMetadata: sill resolveWithNewModule yargs@17.1.1 ch


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://104.155.183.3:8501

npx: installed 22 in 4.448s
your url is: https://calm-birds-dress-104-155-183-3.loca.lt
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
Progress: |██████████████████████████████████████████████████| 100.0% CompleteDownloading recognition model, please wait. This may take several minutes depending upon your network connection.
Progress: |██████████████████████████████████████████████████| 100.0% Complete